In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [96]:
import pandas as pd

df_temp=pd.read_csv('gdrive/MyDrive/Data/GlobalLandTemperaturesByCountry.csv') #Temperature df
df_co2=pd.read_csv('gdrive/MyDrive/Data/CO2emissionbycountries.csv',encoding= 'unicode_escape') #CO2 df

#df_t=pd.read_csv('gdrive/MyDrive/Data/city_temperature.csv') 

In [ ]:
df_temp

In [ ]:
from datetime import datetime
df_temp['dt']= pd.to_datetime(df_temp['dt'])

df_temp['Year'] = pd.DatetimeIndex(df_temp['dt']).year
df_temp = df_temp.drop(columns=['dt','AverageTemperatureUncertainty'])
df_temp = df_temp[['Year', 'Country', 'AverageTemperature']]

df_temp = df_temp[df_temp['Year'] >= 1950]
df_temp = df_temp.reset_index()
print(df_temp)

In [ ]:
df_temp2=df_temp.fillna(method='ffill')
df_temp2

In [ ]:
result = df_temp2.groupby(['Country','Year'])['AverageTemperature'].mean()

# Display result
type(result)
df_temp3 = result
df_temp3.to_frame().reset_index()
#df_temp3['AverageTemperature'].round(2)
df_temp3

In [ ]:
df_co2.head(5)

In [ ]:
#Dropping data with CO2 emission equal 0
#df= df_co2[df_co2['CO2 emission (Tons)'] != 0]
#11880 
df = df_co2
df = df[df['Year'] >= 1959]

df = df[df['Year'] < 2014]

df.fillna(method='ffill')

df=df.drop(columns=['Code','Calling Code','Area','Population(2022)','% of World','Density(km2)'])
df = df.rename(columns={'CO2 emission (Tons)': 'CO2'})
df

In [101]:
#Left outer join exclusion on country with data surveys starting from 1959
df_final=pd.merge(df,df_temp3,on=['Country','Year'],how="inner",indicator=True)
#df_final=df_final[df_final['_merge']=='left_only']
# resetting the DataFrame index
df_final = df_final.reset_index()

df_final=df_final.drop(columns=['index','_merge'])
df_final = df_final.round(2)
df_final['CO2'] = df_final['CO2'].astype(int)
df_final
df_final.to_csv('data_prepared.csv')
!cp data_prepared.csv "gdrive/MyDrive/Data"